In [1]:
import cv2
import numpy as np
import os
from joblib import load
from skimage.feature import hog

In [10]:
# Abrir el archivo de video
#cap = cv2.VideoCapture('video/crosswalk.avi')
cap = cv2.VideoCapture('video/fourway.avi')

# Definir los puntos de referencia de la ROI para detección de carriles
#ROI_vertices = [(0, 720), (1280, 720), (800, 450), (580, 450)]
height, width = cap.read()[1].shape[:2]
ROI_vertices = [(200, height), (1100,height), (550, 250)]


svm = load('modelo_svm.xml')  # Cargar el modelo entrenado del archivo


def draw_lines(img, lines):
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), thickness=2)


def process_frame(frame):
    hog_accum = []

    resized_frame = cv2.resize(frame, (210, 240))

    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    hog_feature, hog_img = hog(gray,
                                               orientations = 11,
                                               pixels_per_cell = (16,16),
                                               cells_per_block = (2,2),
                                               transform_sqrt = False,
                                               visualize = True,
                                               feature_vector = True)

    hog_accum.append(hog_feature)

    # Ajustar dimensiones de los arrays en pedestrian_hog_accum
    min_shape = min(arr.shape[0] for arr in hog_accum)
    hog_accum_adjusted = [arr[:min_shape] for arr in hog_accum]

    # Calcular el descriptor HOG de la imagen
    #descriptor = hog.compute(gray)

    # Ajustar las dimensiones del descriptor
    #descriptor = descriptor.reshape(1, -1)

    # Convertir el descriptor a tipo CV_32F
    #descriptor = descriptor.astype(np.float32)

    # Realizar la predicción utilizando el modelo SVM
    _, result = svm.predict(hog_accum_adjusted)

    # Obtener las coordenadas del rectángulo del peatón detectado
    x, y, w, h = result.ravel()

    # Dibujar un rectángulo alrededor del peatón detectado
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), thickness=2)

    return frame

# Procesar cada cuadro del video y mostrar la imagen de salida
while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:
        line_frame = process_frame(frame)
        cv2.imshow('Video', line_frame)

        # Salir al presionar la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()


ValueError: X has 1077300 features, but SVC is expecting 2112 features as input.